In [1]:
import json
import openai_helper
from azure.search.documents import SearchClient
import search_helper_alta
output_file_name = "alta_enriched_data.json"
vector_file_name = f"{output_file_name}_with_vectors.json"
with open(vector_file_name, "r") as f:
    aml_index_data_with_vectors = json.loads(f.read())
print(len(aml_index_data_with_vectors))


index_name = "alta_index_with_suggester"

Azure Search EndPoint: https://aisearch-c2u66zp7iicfc.search.windows.net
Index Name: aml_index_2
Azure OpenAI Endpoint: https://anildwaopenaiwestus.openai.azure.com/
Azure OpenAI Large Embedding Deployment: text-embedding-ada-002
Azure OpenAI Small Embedding Deployment: text-embedding-ada-002
Embedding Model Name: text-embedding-ada-002
Azure OpenAI API Version: 2024-06-01
Azure Document Intelligence Endpoint: https://anildwadocintelliwestus2.cognitiveservices.azure.com/
1994


v14758586 -

v14758723 -

v14758175 - 

v16141620

v16144611 -

id-SF0A0166207

v15493450 -

v15492295 -

In [40]:

async def rephrase_user_question(user_question, top_k=10):

    search_rephrase_system_message = """
    You are a search assistant. You need to break up the user question into smaller questions such that
    each question can be used to retrieve context from search engine. 
    The question needs to be broken logically without losing the original intent of the question.
    Provide the broken down questions in the following json format:

    Do not include ``` or ```json in the response. Just provide the json object.
    {
        "original_question": "original question",
        "sub_questions": [
            "sub question 1",
            "sub question 2",
            
        ]
        
    }


    """

    #user_question = "what are the steps for installing netbackup for DB2 and explain each step in detail?"

    llm_response = openai_helper.getOpenAIResp(userQuery=user_question, systemMessage=search_rephrase_system_message)

    user_questions = json.loads(llm_response)["sub_questions"]

    print(f"User Questions = {user_questions}")
    complete_context = ""

    for q in user_questions:
        sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=q, top_k=top_k)
        complete_context += json.dumps(sr)


    print(f"Context Token Count1 = {openai_helper.get_token_count(complete_context)}")

    return complete_context

In [42]:
def get_llm_response(user_query, context):
    system_message = f"""
    You are a Veritas Backup product Search assistant. You need to answer user questions on Veritas Backup products like Netbackup. 
    You only need to answer questions related to Veritas Backup products and using the context provided below. 
    
    Answer the user question completely and accurately without omitting any information. 
    
    If the question cannot be answered using the context provided, you can state that you cannot answer the question.

    <Context>{context}</Context>

    Provide the sources of your answers using the format below:
    [section: section_id]

    Provide detailed and accurate answers to the user question.
    """
    print(context)
    print(f"Context Token Count = {openai_helper.get_token_count(context)}")
    llm_response = openai_helper.getOpenAIResp(userQuery=user_query, systemMessage=system_message)
    print(llm_response)

In [32]:

search_query = "what are the steps for installing netbackup for DB2 and explain each step in detail?"

context = await rephrase_user_question(search_query)
#sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=search_query, top_k=20)
#context = f"<Context>{ json.dumps(sr)} </Context>"
get_llm_response(user_query=search_query, context=context)

Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
Context Token Count1 = 14559
[[{"id": "36389f96-5f9b-47e0-8e12-d766992e244d"}, {"part_title": ""}, {"chapter_title": "Installing NetBackup for DB2"}, {"section_title": "Planning the installation of NetBackup for DB2 "}, {"para": "\n\t\t\t\t shows the major installation steps that are needed to run NetBackup for DB2. Each step contains one or more links to pertinent procedures and concepts.StepActionDescriptionStep 1Verify the installation prerequisites. \n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\thttp://www.netbackup.com/compatibility\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tStep 2Verify that the primary server has a valid license for NetBackup for DB2 and any NetBackup options or add-ons.\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tStep 3Specify a log archive method.\n\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\tStep 4(UNIX) 

In [34]:
search_query = "how to enable a policy to archive the logs in DB2 and give me more information on user archive schedules?"
context = await rephrase_user_question(search_query)
#sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=search_query, top_k=20)
#context = f"<Context>{ json.dumps(sr)} </Context>"
get_llm_response(user_query=search_query, context=context)

Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
Context Token Count1 = 7344
[[{"id": "82a1444a-ae82-4726-bbf0-d5b74d781cb8"}, {"part_title": ""}, {"chapter_title": "Configuring NetBackup for DB2"}, {"section_title": "Configuring a policy to archive the archive logs"}, {"para": "This topic describes how to create a policy to archive the NetBackup DB2 archive log entries in the ARCDIR directory. Follow these instructions if you want to use the user exit program with the ARCFUNC COPY command.When NetBackup performs an archive, it deletes the online files after they are backed up successfully.For more information on user archive schedules, see the NetBackup Administrator's Guide, Volume I.You do not need to perform this procedure if you use the VENDOR method to back up your archive log files.Start the NetBackup web UI.Create a new MS-Windows (Windows) or Standard (UNIX) policy type.Specify the attributes for the policy.On the Schedules tab, create a User archive s

In [43]:
search_query = "how to configure a policy to archive the archive logs in DB2 and what is the additional configuration needed if netbackup installed on more than one node in the DB2 cluster?"
context = await rephrase_user_question(search_query, top_k=20)
#sr = await search_helper_alta.retrieve_search_results(index_name=index_name, search_query=search_query, top_k=20)
#context = f"<Context>{ json.dumps(sr)} </Context>"
get_llm_response(user_query=search_query, context=context)


Using Azure OpenAI: https://anildwaopenaiwestus.openai.azure.com//openai/
User Questions = ['How to configure a policy to archive the archive logs in DB2?', 'What additional configuration is needed if NetBackup is installed on more than one node in the DB2 cluster?']
Context Token Count1 = 19745
[[{"id": "82a1444a-ae82-4726-bbf0-d5b74d781cb8"}, {"part_title": ""}, {"chapter_title": "Configuring NetBackup for DB2"}, {"section_title": "Configuring a policy to archive the archive logs"}, {"para": "This topic describes how to create a policy to archive the NetBackup DB2 archive log entries in the ARCDIR directory. Follow these instructions if you want to use the user exit program with the ARCFUNC COPY command.When NetBackup performs an archive, it deletes the online files after they are backed up successfully.For more information on user archive schedules, see the NetBackup Administrator's Guide, Volume I.You do not need to perform this procedure if you use the VENDOR method to back up you